In [1]:
import pandas as pd
from tqdm import  tqdm

<h1>Préparation des données partie 2</h1>
Dans cette partie nous allons faire la construction du vocabulaire et extraction des caractéristiques

In [2]:
data=pd.read_csv('./data.csv')
X=data['text']
X.head()

0    emailaddr thu aug number numbernumbernumb numb...
1    emailaddr mon sep number numbernumbernumb numb...
2    emailaddr wed sep number numbernumbernumb numb...
3    emailaddr wed aug number numbernumbernumb numb...
4    emailaddr wed oct number numbernumbernumb numb...
Name: text, dtype: object

<h2>1-Construction du Vocabulaire<h2>
<h3>1.1 Géneration du vocabulaire<h3>


In [20]:
def update_word_list(email:str,word_list:dict):
    #Extraire tous les mots de l'email et les mettre dans le dictionnaire vocabulaire.
    #Si le mot existe incrementer le nombre d'apparitions sinon insert le nouveau mot dans le dictionnaire et initialiser la valeur d'apparition a 1

IndentationError: expected an indented block (Temp/ipykernel_16152/3887299459.py, line 3)

In [21]:
word_list=dict() #dictionnaire du vocabulaire sous la forme {mot : nombre d'apparition}
for i in tqdm(range(len(X))):
    update_word_list(X[i],word_list)


  0%|          | 0/9349 [00:00<?, ?it/s]


NameError: name 'update_word_list' is not defined

In [22]:
def build_vocabulary(word_list:dict,k:int)->list:
    vocabulary=list()
    #Construire le vocabulaire en prenant les mots dont le nombre d'apparitions est supérieure à 'k'
    #Note : Faire plusieurs tests sur k pour avoir un dictionnaire entre 10000 et 50000
    word_counts = {}
    for word in word_list:
        tokens = clean_email(word)
        for token in tokens:
            if token not in word_counts:
                word_counts[token] = 1
            else:
                word_counts[token] += 1

    # Chercher le bon k pour avoir un dictionnaire entre 10000 et 50000
    for i in range(1, k+1):
        vocab = [word for word, count in word_counts.items() if count >=i ]
        if len(vocab) >= 10000 and len(vocab) <= 50000:
            vocabulary = vocab
            break

    return vocabularyfor word in word_list:
        tokens = clean_email(word_list)
        vocabulary.update(tokens)
    return  vocabulary

In [19]:
#Génération du vocabulaire dans un fichier
k=1
vocabulary=build_vocabulary(word_list, k)
with open('vocab.txt','w') as f:
    for i in vocabulary:
        f.write(f"{i}\n")


<h3>1.2 Indéxation des mots de chaque mail</h3>

In [9]:
def get_vocabulary(filename:str)->list:
    with open(filename, 'r') as file:
        # lire le fichier en tant que texte
        text = file.read()
        # convertir le texte en liste de mots
        vocabulary_list = text.split()
    return vocabulary_list

In [10]:
def indexation(vocabList:list,email:str)->list:
     index=list()
    #Découper l'email en tableau de mot
     words = email.split()
    #Parcourir chaque mot dans l'email et trouver son index dans la liste de vocabulaire
     for word in words:
        if word in vocabList:
            #Ajouter l'index du mot dans la liste d'index de mots
            index.append(vocabList.index(word))
     return index

In [11]:
#just a random test
# test_str = "sing hello to the world eat pizza and chocolate enjoy the sunshine read a book and watch a movie"
# vocabList = get_vocabulary('test.txt')
# indexation(vocabList,test_str)

In [12]:
def generateAllIndexes(vocabList:list,emails)->list:
    list_index=list()
    for i in tqdm(range(len(emails))):
        list_index.append(indexation(vocabulary,X[i]))
    return list_index

<h2>Extraction des caractéristiques</h2>
Représentation des caractéristiques par comptage

In [13]:
def extract_car(vocabList:list,index:list)->list:
    caracteristic_vect=list()
    #Utiliser l'index créé dans l'étape d'indexation pour remplir le vecteur des caractéristique la taille du vecteur est eagle au nombre de mots du vocabulaire et qui sera initialiser à 0 et à chaque apparition du i eme mot faire la mise a jour du vecteur caracteristiques
    return caracteristic_vect

In [14]:
def extract_car_emails(vocabList:list,emails:list)->list:
    allEmailsCarac=list()
    for i in tqdm(range(len(X=emails))):
        allEmailsCarac.append(extract_car(vocabList,emails[i]))